### Guided Project: Analyzing CIA Factbook Data Using SQL

In this project, I'll work with data from the [CIA World Factbook](https://www.cia.gov/the-world-factbook/), a compendium of statistics about all of the countries on Earth. The Factbook contains demographic information like the following:
- `population` — the global population.
- `population_growth` — the annual population growth rate, as a percentage.
- `area` — the total land and water area.

In this project, I'll use SQL in Jupyter Notebook to analyze data from this database. If you want to work on this project locally, you can download the SQLite factbook.db database [here](https://dsserver-prod-resources-1.s3.amazonaws.com/257/factbook.db)

The default Anaconda installation includes Jupyter and several other useful tools.

I'll also need ipython-sql which I'll install by running the following code:

In [1]:
!conda install -yc conda-forge ipython-sql

Solving environment: ...working... done

# All requested packages already installed.



I'll use the following code to connect our Jupyter Notebook to our database file:

__NB:__ The database file should be in the same folder as the notebook file

In [2]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

Just to ensure that my connection was correct, I'll write a query to return all tables in the database

In [3]:
%%sql
SELECT *
FROM sqlite_master
WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"



Now lets explore the contents of the facts table as this is the table which contains the data I want to analyse

In [4]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


From this result, I can see teh different columns in the dataset and can probably even deduce what they correspond to. But for clarity, I'll describe each column and what they correspond to.

- `id` - Serial incremental identifier for table
- `code` -Country code
- `name` - Country name
- `area` - Total land and water area in square kilometers
- `area_land` - Total land area in square kilometers
- `area_water` - Total water area in square kilometers
- `population` - Total population of country
- `population_growth` - Annual population growth as a percentage
- `birth_rate` - Annual birth rate, or the number of births per year per 1,000 people.
- `death_rate` - Annual death rate, or the number of deaths per year per 1,000 people.
- `migration_rate` - - Annual migration rate

**Now, I'll write a query to determine the minimum and maximum population and also the fastest and slowest growing population**

In [5]:
%%sql
SELECT MIN(population) AS 'Minimum Population', 
    MAX(population) AS 'Maximum Population',
    MIN(population_growth) AS 'Minimum Population Growth',
    MAX(population_growth) AS 'Maximum Population Growth'
FROM facts

 * sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,7256490011,0.0,4.02


*Oh wow, who would've thought there was a country with __Zero__ population right? And another country with over 7 billion people yeah? __Hmmmmmmmmmm, Is that not like the population of the whole world???__*

You know what, let's see which countries these are.

In [7]:
%%sql
SELECT name, population
FROM facts
WHERE population in (SELECT MIN(population) FROM facts)

 * sqlite:///factbook.db
Done.


name,population
Antarctica,0


Oh, It obviously had to be Antarctica. Now let's see the country with the highest population

In [8]:
%%sql
SELECT name, population
FROM facts
WHERE population in (SELECT MAX(population) FROM facts)

 * sqlite:///factbook.db
Done.


name,population
World,7256490011


Figures. World is indicated in my dataset as a country.

I'm going to recalculate max and min populations again but this time, I'll exempt the row that contains World. ANd yes, I'll have to exempt world from all my queries from now on

In [9]:
%%sql
SELECT MIN(population) AS 'Minimum Population', 
    MAX(population) AS 'Maximum Population',
    MIN(population_growth) AS 'Minimum Population Growth',
    MAX(population_growth) AS 'Maximum Population Growth'
FROM facts
WHERE name NOT IN ('World')

 * sqlite:///factbook.db
Done.


Minimum Population,Maximum Population,Minimum Population Growth,Maximum Population Growth
0,1367485388,0.0,4.02


Okay, I think its fairer now. 1.3 billion is the highest. I'm guessing that should be China or India. Will probably confirm that later.

For now, I want to calculate the average population of all the countries and also average total surface area

In [11]:
%%sql
SELECT ROUND(AVG(population), 2) AS 'Average Population', 
    ROUND(AVG(area), 3) AS 'Average Area'
FROM facts
WHERE name NOT IN ('World')

 * sqlite:///factbook.db
Done.


Average Population,Average Area
32242666.57,555093.546


Great. Okay, I'm actually curious.

Let me just confirm the most populous country in the world before moving on, lol.

In [12]:
%%sql
SELECT name, population
FROM facts
WHERE population in (SELECT MAX(population) FROM facts WHERE name NOT IN ('World'))

 * sqlite:///factbook.db
Done.


name,population
China,1367485388


I knew it. China. It just had to be. No wonder they are the world manufacturing power.

**_Okay, curiosity satisfied_**

Who else noticed that the average population is quite small. Just over 32 milion. 
I mean, considering the fact that we have the likes of China in this list, tehre has to be an awful lot of countries with really low population to hget this kind of average right. Cos, I mean Nigeria my country even has over 200 million people.

The average area also looks kinda small I think. That's like half of Nigeria's or 60% give or take. I'm really curious to know which countries are this small.

You know what, lets find the countries that have above average values for population and below average values for area

In [16]:
%%sql
SELECT name, population
FROM facts
WHERE population > (SELECT ROUND(AVG(population), 2) FROM facts WHERE name NOT IN ('World'))
AND name NOT IN ('World')

 * sqlite:///factbook.db
Done.


name,population
Afghanistan,32564342
Algeria,39542166
Argentina,43431886
Bangladesh,168957745
Brazil,204259812
Burma,56320206
Canada,35099836
China,1367485388
Colombia,46736728
"Congo, Democratic Republic of the",79375136


_Oh wow, they are actually just a handful_

In [17]:
%%sql
SELECT name, population
FROM facts
WHERE area < (SELECT ROUND(AVG(area), 3) FROM facts WHERE name NOT IN ('World'))
AND name NOT IN ('World')

 * sqlite:///factbook.db
Done.


name,population
Albania,3029278
Andorra,85580
Antigua and Barbuda,92436
Armenia,3056382
Austria,8665550
Azerbaijan,9780780
"Bahamas, The",324597
Bahrain,1346613
Bangladesh,168957745
Barbados,290604


_Okay, this is a loooooot. And there are reallllly small countries like **Cocos Island**, **Vatican city**, etc_

Okay, let's see which country as the highest population growth rate

In [22]:
%%sql
SELECT name, population_growth
FROM facts
WHERE population_growth IN (SELECT MAX(population_growth) FROM facts WHERE name NOT IN ('World'))

 * sqlite:///factbook.db
Done.


name,population_growth
South Sudan,4.02


Okay, this one came as a surprise. Ofcourse I expected it to be in Africa. But for some reason, I did not expect it to be South Sudan.

Next, I want to see which country has more water than land. I've heard there are countries that are mostly water than land.

So first, let's find the ratio of water to land for all the countries yeah


In [32]:
%%sql
SELECT name, area_land, area_water, ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 4) AS 'Ratio'
FROM facts
WHERE name != 'World'

 * sqlite:///factbook.db
Done.


name,area_land,area_water,Ratio
Afghanistan,652230,0,0.0
Albania,27398,1350,0.0493
Algeria,2381741,0,0.0
Andorra,468,0,0.0
Angola,1246700,0,0.0
Antigua and Barbuda,442,0,0.0
Argentina,2736690,43710,0.016
Armenia,28203,1540,0.0546
Australia,7682300,58920,0.0077
Austria,82445,1426,0.0173


From this, the values for the OCeans are empty so it would seem that all the countries here have more land area than water. but let's still confirm that.


In [33]:
%%sql
SELECT name, area_land, area_water, ROUND(CAST(area_water AS Float)/CAST(area_land AS Float), 4) AS 'Ratio'
FROM facts
WHERE name != 'World'
AND Ratio > 1

 * sqlite:///factbook.db
Done.


name,area_land,area_water,Ratio
British Indian Ocean Territory,60,54340,905.6667
Virgin Islands,346,1564,4.5202


Okay, So we have just two countries with a higher water area than land area. `British Indian Ocean Territory` stands out though. Is that ven an Island or a complete water body, lol.	

Now let's explore birth and death rate. let's see which countries have a higher death rate than birth rate cos that would have to be abnormal yeah.

In [34]:
%%sql
SELECT name, birth_rate, death_rate
FROM facts
WHERE name != 'World'
AND death_rate > birth_rate

 * sqlite:///factbook.db
Done.


name,birth_rate,death_rate
Austria,9.41,9.42
Belarus,10.7,13.36
Bosnia and Herzegovina,8.87,9.75
Bulgaria,8.92,14.44
Croatia,9.45,12.18
Czech Republic,9.63,10.34
Estonia,10.51,12.4
Germany,8.47,11.42
Greece,8.66,11.09
Hungary,9.16,12.73


_**WHATTTTT???**_

Okay, this is too much. Germany, Japan, Italy???? I'll need to research this to find out why. This should not be.

DFinally, I want to see which countries have the highest population to area ratio. Let's see which countries we can counsider over populated and **probably contract Thanos to snap a finger at them. lmao**


In [39]:
%%sql
SELECT name, population, area , ROUND(CAST(population AS Float)/area, 2) AS population_area_ratio
FROM facts
WHERE name != 'World'
ORDER BY population_area_ratio DESC

 * sqlite:///factbook.db
Done.


name,population,area,population_area_ratio
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


I think this is too much so let me limit the results to the first 30 entries 

In [40]:
%%sql
SELECT name, population, area , ROUND(CAST(population AS Float)/area, 2) AS population_area_ratio
FROM facts
WHERE name != 'World'
ORDER BY population_area_ratio DESC
LIMIT 30

 * sqlite:///factbook.db
Done.


name,population,area,population_area_ratio
Macau,592731,28,21168.96
Monaco,30535,2,15267.5
Singapore,5674472,697,8141.28
Hong Kong,7141106,1108,6445.04
Gaza Strip,1869055,360,5191.82
Gibraltar,29258,6,4876.33
Bahrain,1346613,760,1771.86
Maldives,393253,298,1319.64
Malta,413965,316,1310.02
Bermuda,70196,54,1299.93


I really don't care what people say but you see Macau and Monaco, definitely needs a snap of Thanos' fingers. lol.

#### Thank you for following me through this journey and story. Your company is well appreciated.

### THE END